In [15]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import math
import os

# (생략된 데이터 로딩 및 제너레이터 부분은 기존 코드와 동일)
# train_dir, val_dir 등은 이미 정의되어 있다고 가정

# 2) 데이터 경로 (기존과 동일)
train_dir = "dataset/train"
val_dir   = "dataset/test"

# 3) 하이퍼파라미터 (기존과 동일)
IMG_SIZE   = (224, 224)
BATCH_SIZE = 16
NUM_CLASSES = 3

# 4) ImageDataGenerator (기존과 동일)
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.1,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    brightness_range=[0.6, 1.4],
    fill_mode='reflect'
)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    directory   = train_dir,
    target_size = IMG_SIZE,
    batch_size  = BATCH_SIZE,
    class_mode  = "categorical",
    shuffle     = True,
    seed        = 42
)
validation_generator = val_datagen.flow_from_directory(
    directory   = val_dir,
    target_size = IMG_SIZE,
    batch_size  = BATCH_SIZE,
    class_mode  = "categorical",
    shuffle     = False
)

num_train = train_generator.samples
num_val   = validation_generator.samples
steps_per_epoch    = math.ceil(num_train / BATCH_SIZE)
validation_steps = math.ceil(num_val    / BATCH_SIZE)

print(f"Train 이미지: {num_train}장 → steps_per_epoch: {steps_per_epoch}")
print(f"Val   이미지: {num_val}장 → validation_steps: {validation_steps}")

# ----------------------------------------------------------------------
# 🌟🌟🌟 미세 조정 시작 부분 (수정) 🌟🌟🌟
# ----------------------------------------------------------------------

print("\n--- 1단계: 헤드(Head)만 훈련된 최적 모델 불러오기 ---")
try:
    # 이전에 저장된 모델을 불러옵니다. 이 모델에는 ResNet50의 모든 레이어와 추가된 헤드가 포함되어 있습니다.
    model = tf.keras.models.load_model('best_model.h5')
    print("✅ 'best_model.h5' 모델을 성공적으로 불러왔습니다.")
except Exception as e:
    print(f"❌ 'best_model.h5'를 불러오는 데 실패했습니다. 에러: {e}")
    print("초기 헤드 훈련이 먼저 성공적으로 완료되고 'best_model.h5'가 저장되어 있는지 확인하세요.")
    exit() # best_model.h5가 없으면 미세 조정을 진행할 수 없으므로 종료

print("\n--- 2단계: 기본 모델 레이어 잠금 해제 (Fine-tuning) ---")

# 모든 레이어를 먼저 학습 불가능하게 설정 (안전한 재설정)
# 헤드 레이어(avg_pool, dropout, predictions)는 제외하고 ResNet50의 모든 레이어를 고정합니다.
# 이렇게 해야 미세 조정할 레이어만 정확히 풀어줄 수 있습니다.
for layer in model.layers:
    if "avg_pool" in layer.name or "dropout" in layer.name or "predictions" in layer.name:
        layer.trainable = True # 헤드 레이어는 학습 가능하게 유지
    else:
        layer.trainable = False # 그 외의 모든 ResNet50 레이어는 일단 고정


# 이제 ResNet50의 특정 블록부터 학습 가능하게 설정합니다.
# 일반적으로 'conv4_block' 또는 'conv5_block'부터 시작합니다.
# 제공된 `Existing layers are: [...]` 목록을 참조하여 정확한 시작 레이어 이름을 사용해야 합니다.
# 예시: 'conv4_block1'의 첫 번째 컨볼루션 레이어부터 학습 가능하게 설정
# model.summary()를 통해 정확한 이름을 확인하세요.
# 여기서는 'conv4_block1_1_conv'가 가장 첫 컨볼루션 레이어라고 가정합니다. (일반적인 ResNet 구조)
# 만약 'conv4_block1_0_conv' 같은 것이 있다면 그 이름을 사용하세요.

unfreeze_from_layer_name = 'conv4_block1_1_conv'
 # 이 이름부터 학습 가능하게 설정 (확인 필요)
set_trainable = False

for layer in model.layers:
    if unfreeze_from_layer_name in layer.name:
        set_trainable = True # 해당 레이어를 발견하면 그 이후 레이어들을 학습 가능하게 전환 시작

    if set_trainable:
        layer.trainable = True
        # print(f"Unfrozen: {layer.name}") # 디버깅용
    # else:
        # print(f"Frozen: {layer.name}") # 디버깅용


print(f"✅ '{unfreeze_from_layer_name}'부터 시작하는 ResNet50 베이스 모델의 레이어를 잠금 해제했습니다.")


print("\n--- 3단계: 아주 낮은 학습률로 모델 재컴파일 ---")
# 미세 조정을 위해 매우 작은 학습률 사용 (예: 1e-5)
# 이전 단계에서 Adam 옵티마이저의 상태가 저장되었을 수 있으므로, 새로운 옵티마이저 인스턴스를 생성하는 것이 좋습니다.
model.compile(optimizer=Adam(1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("\n--- 미세 조정 후 모델 요약 (훈련 가능한 파라미터 확인) ---")
model.summary() # 훈련 가능한 파라미터 수 확인


print("\n--- 4단계: 모델 미세 조정 훈련 시작 ---")
# 콜백 설정 (기존과 동일하게 사용하거나 필요시 patience를 늘릴 수 있음)
early_stop_fine_tune = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, verbose=1) # patience 늘려볼 수 있음
reduce_lr_fine_tune  = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1, min_lr=1e-9) # min_lr 더 낮게 설정
checkpoint_fine_tune = ModelCheckpoint(filepath='best_fine_tuned_model.h5', monitor='val_loss', save_best_only=True, verbose=1)


# 미세 조정을 위한 훈련 (더 많은 에포크 필요)
history_fine_tune = model.fit(
    train_generator,
    steps_per_epoch    = steps_per_epoch,
    validation_data    = validation_generator,
    validation_steps = validation_steps,
    epochs             = 100, # 미세 조정을 위해 충분히 많은 에포크 설정
    callbacks          = [early_stop_fine_tune, reduce_lr_fine_tune, checkpoint_fine_tune]
)

print("\n--- 5단계: 미세 조정된 모델 최종 저장 ---")
model.save("final_fine_tuned_model.h5")
print("✅ 미세 조정된 최종 가중치를 final_fine_tuned_model.h5 로 저장했습니다.")

Found 214 images belonging to 3 classes.


Found 174 images belonging to 3 classes.
Train 이미지: 214장 → steps_per_epoch: 14
Val   이미지: 174장 → validation_steps: 11

--- 1단계: 헤드(Head)만 훈련된 최적 모델 불러오기 ---


✅ 'best_model.h5' 모델을 성공적으로 불러왔습니다.

--- 2단계: 기본 모델 레이어 잠금 해제 (Fine-tuning) ---
✅ 'conv4_block1_1_conv'부터 시작하는 ResNet50 베이스 모델의 레이어를 잠금 해제했습니다.

--- 3단계: 아주 낮은 학습률로 모델 재컴파일 ---

--- 미세 조정 후 모델 요약 (훈련 가능한 파라미터 확인) ---


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

...

14/14 ━━━━━━━━━━━━━━━━━━━━ 53s 4s/step - accuracy: 0.5243 - loss: 1.0578 - val_accuracy: 0.3103 - val_loss: 1.1261 - learning_rate: 1.0000e-04
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4567 - loss: 1.0902
Epoch 7: val_loss improved from 1.12613 to 1.02518, saving model to best_fine_tuned_model.h5


14/14 ━━━━━━━━━━━━━━━━━━━━ 52s 4s/step - accuracy: 0.4599 - loss: 1.0863 - val_accuracy: 0.4828 - val_loss: 1.0252 - learning_rate: 1.0000e-04
Epoch 8/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4850 - loss: 0.9764
Epoch 8: val_loss did not improve from 1.02518
14/14 ━━━━━━━━━━━━━━━━━━━━ 52s 4s/step - accuracy: 0.4879 - loss: 0.9765 - val_accuracy: 0.4885 - val_loss: 1.0534 - learning_rate: 1.0000e-04
Epoch 9/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6443 - loss: 0.9022
Epoch 9: val_loss did not improve from 1.02518
14/14 ━━━━━━━━━━━━━━━━━━━━ 52s 4s/step - accuracy: 0.6440 - loss: 0.9088 - val_accuracy: 0.4540 - val_loss: 1.0696 - learning_rate: 1.0000e-04
Epoch 10/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5158 - loss: 0.9820
Epoch 10: val_loss did not improve from 1.02518
14/14 ━━━━━━━━━━━━━━━━━━━━ 52s 4s/step - accuracy: 0.5147 - loss: 0.9855 - val_accuracy: 0.4368 - val_loss: 1.0597 - learning_rate: 1.0000e-04
Epoch 11/100
14/14 ━━━━━━━━━━━━━━

14/14 ━━━━━━━━━━━━━━━━━━━━ 57s 4s/step - accuracy: 0.6187 - loss: 0.8608 - val_accuracy: 0.4655 - val_loss: 1.0225 - learning_rate: 5.0000e-05
Epoch 14/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6030 - loss: 0.8068
Epoch 14: val_loss did not improve from 1.02252
14/14 ━━━━━━━━━━━━━━━━━━━━ 53s 4s/step - accuracy: 0.6011 - loss: 0.8110 - val_accuracy: 0.4885 - val_loss: 1.1375 - learning_rate: 5.0000e-05
Epoch 15/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6447 - loss: 0.7882
Epoch 15: val_loss did not improve from 1.02252
14/14 ━━━━━━━━━━━━━━━━━━━━ 52s 4s/step - accuracy: 0.6457 - loss: 0.7876 - val_accuracy: 0.4885 - val_loss: 1.0717 - learning_rate: 5.0000e-05
Epoch 16/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6410 - loss: 0.7928
Epoch 16: val_loss did not improve from 1.02252
14/14 ━━━━━━━━━━━━━━━━━━━━ 52s 4s/step - accuracy: 0.6397 - loss: 0.7952 - val_accuracy: 0.5230 - val_loss: 1.0344 - learning_rate: 5.0000e-05
Epoch 17/100
14/14 ━━━━━━━━━━

14/14 ━━━━━━━━━━━━━━━━━━━━ 52s 4s/step - accuracy: 0.6536 - loss: 0.7341 - val_accuracy: 0.4885 - val_loss: 1.0054 - learning_rate: 5.0000e-05
Epoch 18/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7267 - loss: 0.7008
Epoch 18: val_loss did not improve from 1.00540
14/14 ━━━━━━━━━━━━━━━━━━━━ 52s 4s/step - accuracy: 0.7247 - loss: 0.7027 - val_accuracy: 0.4655 - val_loss: 1.4696 - learning_rate: 5.0000e-05
Epoch 19/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5467 - loss: 0.8734
Epoch 19: val_loss did not improve from 1.00540
14/14 ━━━━━━━━━━━━━━━━━━━━ 53s 4s/step - accuracy: 0.5479 - loss: 0.8766 - val_accuracy: 0.5115 - val_loss: 1.0231 - learning_rate: 5.0000e-05
Epoch 20/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5593 - loss: 0.8606
Epoch 20: val_loss did not improve from 1.00540
14/14 ━━━━━━━━━━━━━━━━━━━━ 53s 4s/step - accuracy: 0.5575 - loss: 0.8649 - val_accuracy: 0.4713 - val_loss: 1.0539 - learning_rate: 5.0000e-05
Epoch 21/100
14/14 ━━━━━━━━━━


--- 5단계: 미세 조정된 모델 최종 저장 ---
✅ 미세 조정된 최종 가중치를 final_fine_tuned_model.h5 로 저장했습니다.


In [16]:
import os

base_dir = "dataset/train"
for class_name in ["heavy", "medium", "clean"]:
    folder = os.path.join(base_dir, class_name)
    files = os.listdir(folder)
    print(f"{class_name}: {len(files)}장")


heavy: 62장
medium: 72장
clean: 80장
